In [22]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Dense, Activation, Input, Embedding, GlobalAveragePooling1D

In [33]:
imdb = tensorflow.keras.datasets.imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=2000)

In [34]:
word_index = imdb.get_word_index()

# The first indices are reserved
word_index = {k:(v+3) for k,v in word_index.items()} 
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2  # unknown
word_index["<UNUSED>"] = 3

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

In [47]:
decode_review(train_data[0])

"<START> this film was just brilliant casting location scenery story direction <UNK> really <UNK> the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same <UNK> island as myself so i loved the fact there was a real <UNK> with this film the witty <UNK> throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the <UNK> <UNK> was amazing really <UNK> at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little <UNK> that played the <UNK> of <UNK> and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all <UNK> up are such a big <UNK> for the whole film but these children are amazing and should be <UNK> for what they have done don

In [36]:
train_data = keras.preprocessing.sequence.pad_sequences(train_data,
                                                        value=word_index["<PAD>"],
                                                        padding='post',
                                                        maxlen=256)

test_data = keras.preprocessing.sequence.pad_sequences(test_data,
                                                       value=word_index["<PAD>"],
                                                       padding='post',
                                                       maxlen=256)

In [41]:
# input shape is the vocabulary count used for the movie reviews (10,000 words)
vocab_size = 2000

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation=tf.nn.relu))
model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 16)          32000     
_________________________________________________________________
global_average_pooling1d_3 ( (None, 16)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 17        
Total params: 32,289
Trainable params: 32,289
Non-trainable params: 0
_________________________________________________________________


In [42]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [43]:
x_val = train_data[:10000]
partial_x_train = train_data[10000:]

y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]

In [44]:
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=5,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

/home/bernardocordeiro/anaconda3/envs/tensorflow/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 15000 samples, validate on 10000 samples
Epoch 1/5
15000/15000 [==============================] - 1s 47us/step - loss: 0.6927 - acc: 0.5868 - val_loss: 0.6920 - val_acc: 0.5833
Epoch 2/5
15000/15000 [==============================] - 0s 27us/step - loss: 0.6903 - acc: 0.6327 - val_loss: 0.6878 - val_acc: 0.6925
Epoch 3/5
15000/15000 [==============================] - 0s 32us/step - loss: 0.6831 - acc: 0.7189 - val_loss: 0.6777 - val_acc: 0.6885
Epoch 4/5
15000/15000 [==============================] - 0s 32us/step - loss: 0.6673 - acc: 0.7365 - val_loss: 0.6568 - val_acc: 0.7057
Epoch 5/5
15000/15000 [==============================] - 0s 29us/step - loss: 0.6396 - acc: 0.7493 - val_loss: 0.6263 - val_acc: 0.7513


In [48]:
saver = tf.train.import_meta_graph('BERT-English/bert_model.ckpt.meta')

In [59]:
with tf.Session() as sess:
    #saver = tf.train.Saver(save_relative_paths=True)
    saver = tf.train.import_meta_graph('bert_model.ckpt.meta')
    saver.restore(sess, tf.train.latest_checkpoint('bert_model.ckpt.data-00000-of-00001'))

ValueError: Can't load save_path when it is None.